In [1]:
import numpy as np


from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt #Para graficar
from skimage.io import imshow #Para graficar las imagenes
import tensorflow as tf


import os
import glob
##from PIL import Image
import cv2

#import pyheif

import matplotlib.pyplot as plt

In [2]:
#root_path = "C:/MaestriaDocs/DMA - Proyecto/caras_grupo4/"
root_path = "../"

directorio_personas = root_path + "Caras_cortadas"

Obtenemos de las carpetas originales las fotos

In [3]:
from PCA_funciones import matriz_fotos_desde_carpeta

image_matrix, image_person = matriz_fotos_desde_carpeta(directorio_personas)

Carpeta: Abel
Carpeta: Carlos
Carpeta: Federico G
Carpeta: Federico R
Carpeta: Florencia
Carpeta: Franco A
Carpeta: Franco S
Carpeta: Gerard
Carpeta: Gustavo
Carpeta: Joaquin
Carpeta: Juan
Carpeta: Lautaro
Carpeta: Lisandro
Carpeta: Marco
Carpeta: Matias
Carpeta: Natalia
Carpeta: Noelia
Carpeta: Paola
Carpeta: Victorio


## Aplicamos Backpropagation
Transformamos las imagenes, con la cantidad de copoenentes 

In [4]:
from sklearn.decomposition import PCA
import pickle

# Cargar los componentes PCA guardados
ruta_pca = root_path + "PCA/Componentes_pca.npy" #lo tenemos en memoria pero lo leemos nuevamente por si separamos el codigo
pca_components = np.load(ruta_pca)

# Cargar el escalador
ruta_scaler = root_path + 'PCA/scaler.pkl' #
with open(ruta_scaler, 'rb') as f:
    scaler = pickle.load(f)

# Función para transformar una imagen con un rango específico de componentes PCA
def transformar_imagen_pca(imagen, scaler, pca_components, num_componentes=50, inicio_componente=0):
    # Estandarizar la imagen
    imagen_estandarizada = scaler.transform(imagen)
    
    # Seleccionar el rango de componentes
    componentes_seleccionados = pca_components[inicio_componente:inicio_componente + num_componentes]
    
    # Realizar la proyección manualmente
    imagen_pca = np.dot(imagen_estandarizada, componentes_seleccionados.T)
    return imagen_pca


# Especificar la cantidad de componentes a utilizar y el componente de inicio
num_componentes = 60
inicio_componente = 2 #inicia a partir de la 3ta componente

# Transformar la nueva imagen utilizando los componentes especificados
imagenes_pca_reducidas = transformar_imagen_pca( image_matrix, scaler, pca_components, num_componentes, inicio_componente)

print(imagenes_pca_reducidas)

imagenes_pca_reducidas.shape
personas = np.unique(image_person)
image_person #(vector que tiene el nombre de la persona de cada foto)
personas
personas.shape
cantidad_personas = len(personas)
print(cantidad_personas) 
cantidad_fotos = len(image_person)
cantidad_fotos
# salida_lista = []  # Array vacío para almacenar las listas de 0 y 1
# cantidad_personas = len(personas) #podiramos hacer un elemento menos pero por ahora lo dejamos en el total
# # Recorrer cada elemento en "image_person"
# for imagen_persona in image_person:
#   lista_persona = [0] * cantidad_personas   # Inicializar lista con 18 ceros
  
#   # Buscar el índice del nombre de la imagen en "personas"
#  # Encontrar el índice de la persona
#   indice_persona =  np.where(personas == imagen_persona)[0]

#   # Si se encuentra la persona, actualizar la lista
#   if len(indice_persona) > 0:
#     lista_persona[indice_persona[0]] = 1
  
#   # Actualizar el elemento correspondiente en la lista con 1

  
#   # Agregar la lista a Y
#   salida_lista.append(lista_persona)

# salida_lista = np.array(salida_lista) 
# salida_lista

imagenes_pca_reducidas[0]
X=imagenes_pca_reducidas
X.shape
from sklearn.preprocessing import  LabelBinarizer
# Convertir los valores de la var categórica en números
#salida_lista = LabelBinarizer().fit_transform(image_person) #convierte los nombres (var categorica) en vextores con un 1 en la posicion del nombre
#correspondiente, tiene un renglon por foto y el vector una columna por nombre solo un 1 en el nombre de la persona de la foto


label_binarizer = LabelBinarizer()
salida_lista = label_binarizer.fit_transform(image_person)
salida_lista
salida_lista.shape
X = imagenes_pca_reducidas
Y = salida_lista

# Normalización de las entradas
"""mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - mean) / std
"""
# Convierte la lista a un arreglo NumPy
#salida_lista = nombres_personas
salida = np.array(salida_lista)
Y = salida #salida.reshape(len(X),1)

epoch_limit = 30


# Semilla para reproducibilidad
np.random.seed(1021) #Establece la semilla para las funciones aleatorias de numpy.  
tf.random.set_seed(1021) #Establece la semilla para las funciones aleatorias de TensorFlow, asegurando que los pesos iniciales de la red neuronal y 4cualquier otra operación aleatoria en TensorFlow sean reproducibles.
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1021)

[[ 7.22160860e+00 -7.80849051e+00 -2.07612021e-01 ... -7.16897218e-01
  -2.28025618e-01 -3.38966655e-01]
 [-7.84286624e+00 -1.12024841e+01  4.89598300e-01 ... -7.43509551e-01
  -6.18654161e-01  2.88035939e-01]
 [-6.55630242e+00 -7.80088794e+00 -1.49179444e+00 ... -1.37140978e+00
  -2.96450608e-01  2.05965925e-02]
 ...
 [ 3.55657307e+00  6.49301206e-01 -6.23891347e+00 ... -6.32500626e-02
   3.18870940e-01 -6.26304974e-01]
 [-2.65755973e+00 -4.44534213e-03 -8.82886566e+00 ...  2.92722209e-01
  -9.78650328e-02 -2.65089331e-01]
 [-2.62864833e+00  8.66772857e-01 -8.11920552e+00 ...  1.20877517e-01
  -1.40180893e-02 -1.42284246e-01]]
19


## usando tensor flow

In [5]:
print(X_train.shape)
X_test.shape

(964, 60)


(242, 60)

In [6]:
print(Y_train.shape)
Y_test.shape

(964, 19)


(242, 19)

In [7]:
import tensorflow as tf


# Definir el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(num_componentes,)),  # Capa de entrada con 60 características - Cantidad de componentes PCA
    tf.keras.layers.Dense(128, activation='relu'),  # Capa oculta con 128 neuronas y función de activación ReLu (trasmite la infoamción generada por la combinación lineal de los pesos y las entredas, especificamente ReLu permite la propagación hacia atrás efectiva)
                                                    # ReLu genera una salida igual a cero cuando la entrada (z) sea negativa, y una salida igual a la entrada cuando está última es positiva. 
    tf.keras.layers.Dense(cantidad_personas, activation='softmax') # Capa de salida con 18 neuronas y función de activación Softmax (La función calcula la distribución de probabilidades del evento sobre "n" eventos diferentes)
                                                    # Convierte los datos en probabilidades y la suma de las probabilidades da como resultado 1.
])

# Compilar / configurar el proceso de entrenamiento del modelo
model.compile(optimizer='adam', # El optimizador ajusta los parámetros de la red (pesos y sesgos) para minimizar la función pérdida.
                                # Tasa de aprendizaje que se adapta a cada parámetro en forma individual en función de las estimaciones del momento y de la magnitud del gradiente
                                # Busca que el modelo se ajuste a los datos de entrenamiento de manera eficiente y efectiva lo que puede mejorar la predicción.
              loss='categorical_crossentropy',  # Función de pérdida para clasificación multiclase. Mide la diferencia de entre las distribucuines de probabilidades predichas por el modelo y las distribuciones reales de las clases.
              metrics=['accuracy']) # Métrica utilizada para evaluar el rendimiento del modelo durante el entrenamiento y la evaluación
                                    # Se una para medir la precisión del modelo (proporción de muestras clasificadas correctamente sobre el total de las muestas)


# Entrenar el modelo
histort=model.fit(X_train, Y_train, epochs=epoch_limit, batch_size=32, validation_data=(X_test, Y_test)) # ANALIZAR GUARDAR LOS RESULTADOS

Epoch 1/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.1037 - loss: 3.7314 - val_accuracy: 0.4339 - val_loss: 2.0386
Epoch 2/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6291 - loss: 1.4308 - val_accuracy: 0.7314 - val_loss: 1.1077
Epoch 3/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8759 - loss: 0.6950 - val_accuracy: 0.8678 - val_loss: 0.7165
Epoch 4/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9405 - loss: 0.4077 - val_accuracy: 0.8967 - val_loss: 0.5352
Epoch 5/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9721 - loss: 0.2721 - val_accuracy: 0.9174 - val_loss: 0.4314
Epoch 6/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9818 - loss: 0.1968 - val_accuracy: 0.9215 - val_loss: 0.3644
Epoch 7/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9859 - loss: 0.1495 - val_accuracy: 0.9298 - val_loss: 0.3180
Epoch 8/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9884 - loss: 0.1174 - val_accuracy: 0.9339 - val_loss:

In [8]:
# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

model.evaluate(X_test, Y_test) #BORRAR

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9778 - loss: 0.1075 
Loss: 0.1503334641456604, Accuracy: 0.9669421315193176
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9778 - loss: 0.1075 


[0.1503334641456604, 0.9669421315193176]

In [9]:
# Obtener las predicciones para el conjunto de prueba
predictions = model.predict(X_test)

# Iterar sobre las predicciones y mostrar el resultado para cada foto
for i in range(len(predictions)):
    # Obtener la predicción para la i-ésima foto
    prediction = predictions[i]
    # Encontrar la etiqueta predicha (la clase con probabilidad más alta)
    predicted_label = np.argmax(prediction) # Identifica la clase que el modelo considera más probable para la imagen
    # Mostrar el resultado
    real = np.argmax(Y_test[i])
    predicted_probability= np.max(prediction)
    print(f"Foto {i+1}: Real = {real}, Predicción = {predicted_label}, Probabillidad = {predicted_probability}, Probabilidades = {prediction}") # nro de foto - la clase con mayor probabilidad - probabilidad de pertenencia a cada clase.


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Foto 1: Real = 4, Predicción = 4, Probabillidad = 0.980628490447998, Probabilidades = [3.4674143e-03 2.4920153e-06 3.5700170e-04 2.3650211e-05 9.8062849e-01
 3.2486198e-06 2.2864737e-05 2.3825476e-03 8.2356906e-05 1.8398508e-05
 3.1653242e-05 2.5956405e-03 9.6898373e-05 1.0728772e-04 6.3858170e-04
 8.3257510e-06 1.6380484e-04 8.1000132e-03 1.2693169e-03]
Foto 2: Real = 11, Predicción = 11, Probabillidad = 0.9972062706947327, Probabilidades = [4.7198282e-06 1.9074614e-08 1.6184284e-05 1.2657305e-04 4.1782376e-05
 4.9503401e-06 4.5203092e-06 2.8078221e-06 1.6497056e-03 2.9313404e-04
 1.5890396e-07 9.9720627e-01 7.1901995e-05 2.5740212e-06 6.7274800e-06
 9.5496718e-05 4.1565235e-04 7.7697132e-06 4.9007376e-05]
Foto 3: Real = 18, Predicción = 18, Probabillidad = 0.9980490207672119, Probabilidades = [1.6057675e-05 4.5266963e-05 5.1264228e-06 4.2239640e-06 3.5105866e-06
 1.3088713e-04 1.2717842e-05 3.7972775e-05 2.1501760e-04 1.1146496e-03
 7.6211459e-06

In [10]:
y_predicted = model.predict(X_test)
y_predicho = np.argmax(y_predicted, axis=1)
y_testeo = np.argmax(Y_test, axis=1).reshape(-1, ) 
error_i = (y_predicho - y_testeo != 0)
error_i

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [11]:
# Cuántos hay
print('En total el modelo presenta {} errores en el dataset de testeo donde hay un total de {} imagenes.'.format(np.sum(error_i), len(X_test)))

En total el modelo presenta 8 errores en el dataset de testeo donde hay un total de 242 imagenes.


In [12]:
import pandas as pd

In [13]:
# Convertir índices de clases a nombres de clases
y_test_names = label_binarizer.inverse_transform(Y_test) #reales de test
y_pred_names = label_binarizer.inverse_transform(y_predicted)


# Crear DataFrame
tabla_resultados = pd.DataFrame({
    'y_predicho': y_pred_names,
    'y_testeo': y_test_names
})
tabla_resultados['Predicción correcta'] = tabla_resultados['y_predicho'] == tabla_resultados['y_testeo']
tabla_resultados

,y_predicho,y_testeo,Predicción correcta
0,Florencia,Florencia,True
1,Lautaro,Lautaro,True
2,Victorio,Victorio,True
3,Paola,Paola,True
4,Franco S,Franco S,True
...,...,...,...
237,Franco S,Franco S,True
238,Carlos,Carlos,True
239,Natalia,Natalia,True
240,Carlos,Carlos,True


## Encontrar el mejor modelo


In [14]:
#pip install scikeras

importamos paquetes

In [15]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

In [16]:
def create_model(optimizer='adam', activation='relu', neurons=128, **kwargs):
    

    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(num_componentes,)),  # Capa de entrada con 60 características - Cantidad de componentes PCA
    tf.keras.layers.Dense(neurons, activation='relu'),  # Capa oculta con 128 neuronas y función de activación ReLu (trasmite la infoamción generada por la combinación lineal de los pesos y las entredas, especificamente ReLu permite la propagación hacia atrás efectiva)
                                                    # ReLu genera una salida igual a cero cuando la entrada (z) sea negativa, y una salida igual a la entrada cuando está última es positiva. 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(cantidad_personas, activation='softmax') # Capa de salida con 18 neuronas y función de activación Softmax (La función calcula la distribución de probabilidades del evento sobre "n" eventos diferentes)
                                                    # Convierte los datos en probabilidades y la suma de las probabilidades da como resultado 1.

    ])
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

Creammos el clasificador

In [17]:
model = KerasClassifier(model=create_model, verbose=0)


Definir hiperparametros

In [18]:
param_grid = {
    'model__optimizer': ['adam', 'sgd'],
    'model__activation': ['relu', 'sigmoid', 'tanh', 'softmax'],
    'model__neurons': [12, 64, 128, 256,512], #neuronas de la capa oculta
    'epochs': [20, 30, 60,70]  # Añadimos el número de épocas como hiperparámetro
}



Buscamos hiperparametros

In [19]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(X_train, Y_train)


Imprimir los mejores hyperparametros

In [ ]:
print("Mejores hiperparámetros:", grid_result.best_params_)

Mejores hiperparámetros: {'epochs': 70, 'model__activation': 'sigmoid', 'model__neurons': 512, 'model__optimizer': 'adam'}


In [ ]:
grid_result

GridSearchCV(cv=3,
             estimator=KerasClassifier(model=<function create_model at 0x0000021E71271B20>, verbose=0),
             param_grid={'epochs': [20, 30, 60, 70],
                         'model__activation': ['relu', 'sigmoid'],
                         'model__neurons': [64, 128, 256, 512],
                         'model__optimizer': ['adam', 'sgd']})

Obtener el mejor modelo

In [ ]:
best_model = grid_result.best_estimator_


Evaluar el modelo en los datos de prueba

In [ ]:
score = best_model.score(X_test, Y_test)
print("Precisión en los datos de prueba:", score)

#best_model.score(X_test, y_test) evalúa el mejor modelo en el conjunto de prueba, devolviendo la precisión del modelo.

Precisión en los datos de prueba: 0.9793388429752066


Precision de datos de prueba

Usando esta matriz, las fórmulas serían:

__Precision:__ Proporción de TP sobre todos los predichos positivos (TP + FP).
__Recall:__ Proporción de TP sobre todos los reales positivos (TP + FN).
__F1 Score:__ Media armónica de precisión y recall.
__Support:__ Cantidad total de ejemplos reales positivos y negativos.

__Precision (Precisión)__
La proporción de verdaderos positivos (TP) entre todos los ejemplos que el modelo ha clasificado como positivos.
Cómo se calcula: 
precision  = TP / (TP + FP)
            donde TP son los verdaderos positivos y FP son los falsos positivos.
Uso: Indica cuán preciso es el modelo cuando predice la clase positiva. Es útil en situaciones donde el costo de un falso positivo es alto.


__Recall (Sensibilidad)__
La proporción de verdaderos positivos entre todos los ejemplos que realmente son positivos.
Cómo se calcula:
recall  = TP / (TP + FN)
        donde TP son los verdaderos positivos y FN son los falsos negativos.
*Uso:* Indica cuán bien el modelo captura los verdaderos positivos. Es útil en situaciones donde el costo de un falso negativo es alto.


__F1 Score__
Es una medida combinada que toma en cuenta tanto la precisión (precision) como la exhaustividad (recall).
Cómo se calcula: Es la media armónica de la precisión y el recall.
Fórmula:
 𝐹1 = 2 × precision × recall / ( precision + recall)
  
*Uso:* Se utiliza para evaluar modelos de clasificación, especialmente cuando hay un desequilibrio entre las clases.


__Support (Soporte)__
Qué mide: El número de ocurrencias de cada clase en el conjunto de datos verdadero.
*Uso:* Proporciona contexto para las métricas de precisión, recall y F1 score, mostrando cuántos ejemplos de cada clase hay en los datos.

In [ ]:
from sklearn.metrics import classification_repor__test, axis=1)

# Imprimir el reporte de clasificación
print(classification_report(y_test_true, y_pred_max))

# Imprimir la matriz de confusión
print(confusion_matrix(y_test_true, y_pred_max))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00         8
           5       1.00      1.00      1.00        18
           6       1.00      1.00      1.00         9
           7       0.94      1.00      0.97        16
           8       1.00      1.00      1.00         7
           9       1.00      0.88      0.94        17
          10       1.00      1.00      1.00        10
          11       0.91      1.00      0.95        20
          12       1.00      1.00      1.00        13
          13       1.00      1.00      1.00        10
          14       1.00      1.00      1.00        15
          15       1.00      1.00      1.00         7
          16       0.93      0.93      0.93        14
          17       0.95    

In [ ]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# Convertir índices de clases a nombres de clases
y_test_names = label_binarizer.inverse_transform(Y_test) #reales de test
y_pred_names = label_binarizer.inverse_transform(y_pred)


# Crear DataFrame
tabla_resultados = pd.DataFrame({
    'y_predicho': y_pred_names,
    'y_testeo': y_test_names
})
tabla_resultados['Predicción correcta'] = tabla_resultados['y_predicho'] == tabla_resultados['y_testeo']
tabla_resultados

,y_predicho,y_testeo,Predicción correcta
0,Florencia,Florencia,True
1,Lautaro,Lautaro,True
2,Victorio,Victorio,True
3,Paola,Paola,True
4,Franco S,Franco S,True
...,...,...,...
237,Franco S,Franco S,True
238,Carlos,Carlos,True
239,Natalia,Natalia,True
240,Carlos,Carlos,True


In [ ]:
#tabla_resultados['Predicción correcta']==false
np.sum(tabla_resultados['Predicción correcta'])/len(tabla_resultados)

0.9793388429752066

In [ ]:
filtro_falsos = tabla_resultados[tabla_resultados['Predicción correcta'] == False]
print(filtro_falsos)

    y_predicho  y_testeo  Predicción correcta
69      Noelia      Abel                False
149    Lautaro   Joaquin                False
177    Lautaro   Joaquin                False
186      Paola    Noelia                False
199     Gerard  Victorio                False
